## TPC 2 - Conversor de MarkDown para HTML

Criar em Python um pequeno conversor de MarkDown para HTML para os elementos descritos na "Basic Syntax" da Cheat Sheet:


In [2]:
import re

### Cabeçalhos: linhas iniciadas por "# texto", ou "## texto" ou "### texto"

In: `# Exemplo`

Out: `<h1>Exemplo</h1>`


In [3]:
exemplo_1 = "# Hello"
exemplo_2 = "## My"
exemplo_3 = "### Friend"

def troca_titulo(linha):
    rc = re.compile(r"^(#{1,3})\s+(.+)")
    m = rc.match(linha)
    if not m:
        return linha
    nivel = len(m.group(1))
    conteudo = m.group(2)
    return f"<h{nivel}>{conteudo}</h{nivel}>"

print(troca_titulo(exemplo_1))
print(troca_titulo(exemplo_2))
print(troca_titulo(exemplo_3))

<h1>Hello</h1>
<h2>My</h2>
<h3>Friend</h3>


### Bold: pedaços de texto entre "**":

In: `Este é um **exemplo** ...`

Out: `Este é um <b>exemplo</b> ...`

### Itálico: pedaços de texto entre "*":

In: `Este é um *exemplo* ...`

Out: `Este é um <i>exemplo</i> ...`

In [4]:
exemplo_1 = "Este é um **exemplo** ..."
exemplo_2 = "Este é um *exemplo* ..."
exemplo_3 = "Este é um *****exemplo*** ..."

def troca_bi(linha):
    # primeiro bold+italico, depois bold, depois italico
    linha = re.sub(r"\*\*\*(.+?)\*\*\*", r"<b><i>\1</i></b>", linha)
    linha = re.sub(r"\*\*(.+?)\*\*", r"<b>\1</b>", linha)
    linha = re.sub(r"\*(.+?)\*", r"<i>\1</i>", linha)
    return linha

print(troca_bi(exemplo_1))
print(troca_bi(exemplo_2))
print(troca_bi(exemplo_3))

Este é um <b>exemplo</b> ...
Este é um <i>exemplo</i> ...
Este é um <b><i>**exemplo</i></b> ...



### Lista numerada:

In:
```
1. Primeiro item
2. Segundo item
3. Terceiro item
```

Out:
```
<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>
```


In [5]:
exemplo = """1. Primeiro item
2. Segundo item
3. Terceiro item"""

def troca_lista(texto):
    linhas = texto.splitlines()
    saida = []
    dentro_ol = False
    dentro_ul = False
    for linha in linhas:
        m_ol = re.match(r"^(\d+)\.\s+(.+)", linha)
        m_ul = re.match(r"^[-*]\s+(.+)", linha)
        if m_ol:
            if not dentro_ol:
                if dentro_ul:
                    saida.append("</ul>")
                    dentro_ul = False
                saida.append("<ol>")
                dentro_ol = True
            saida.append(f"<li>{m_ol.group(2)}</li>")
        elif m_ul:
            if not dentro_ul:
                if dentro_ol:
                    saida.append("</ol>")
                    dentro_ol = False
                saida.append("<ul>")
                dentro_ul = True
            saida.append(f"<li>{m_ul.group(1)}</li>")
        else:
            if dentro_ol:
                saida.append("</ol>")
                dentro_ol = False
            if dentro_ul:
                saida.append("</ul>")
                dentro_ul = False
            saida.append(linha)
    if dentro_ol:
        saida.append("</ol>")
    if dentro_ul:
        saida.append("</ul>")
    return "\n".join(saida)

print(troca_lista(exemplo))

<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>


### Link: [texto](endereço URL)

In: `Como pode ser consultado em [página da UC](http://www.uc.pt)`

Out: `Como pode ser consultado em <a href="http://www.uc.pt">página da UC</a>`



In [6]:
exemplo = r"Como pode ser consultado em [página da UC](http://www.uc.pt)"

def troca_link(linha):
    rc = re.compile(r"\[([^\]]+)\]\(([^)]+)\)")
    return rc.sub(lambda m: f'<a href="{m.group(2)}">{m.group(1)}</a>', linha)

print(troca_link(exemplo))

Como pode ser consultado em <a href="http://www.uc.pt">página da UC</a>


### Imagem: ![texto alternativo](path para a imagem)

In: `Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ...`

Out: `Como se vê na imagem seguinte: <img src="http://www.coellho.com" alt="imagem dum coelho"/> ...`

In [7]:
exemplo = r"Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ..."

def troca_imagem(linha):
    rc = re.compile(r"!\[([^\]]*)\]\(([^)]+)\)")
    return rc.sub(lambda m: f'<img src="{m.group(2)}" alt="{m.group(1)}"/>', linha)

print(troca_imagem(exemplo))

Como se vê na imagem seguinte: <img src="http://www.coellho.com" alt="imagem dum coelho"/> ...


### Conversor

Agora, juntamos todas as funções na função `markdown_html()`

In [8]:
texto = """## Título
Texto em **negrito** e *itálico*
1. Item
2. Outro item
Um [link](http://teste.com)
Uma imagem ![alt](http://img.com/x.png)"""


def markdown_html(texto):
    texto = troca_lista(texto)
    linhas_saida = []
    for linha in texto.splitlines():
        linha = troca_titulo(linha)
        linha = troca_bi(linha)
        linha = troca_link(linha)
        linha = troca_imagem(linha)
        linhas_saida.append(linha)
    return "\n".join(linhas_saida)

print(markdown_html(texto))

<h2>Título</h2>
Texto em <b>negrito</b> e <i>itálico</i>
<ol>
<li>Item</li>
<li>Outro item</li>
</ol>
Um <a href="http://teste.com">link</a>
Uma imagem !<a href="http://img.com/x.png">alt</a>
